## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# from collections import OrderedDict
from keras import backend as K
from keras import optimizers
from keras.callbacks import Callback, EarlyStopping  # , ModelCheckpoint
from keras.layers import Dense, Dropout, Input, Layer
from keras.models import Model, Sequential
from keras.regularizers import l1
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# sys.path.append("../utils")
# from common_functions import find_parent
# from common_functions import leaves_id
# from common_functions import get_list_split_phi_forest
from RTF.utils.common_functions import (get_list_split_phi,
									  get_parents_nodes_leaves_dic,
							  		  print_decision_path)
from RTF.nrf.Neural_Decision_Tree import *
from RTF.nrf.Neural_Random_Forest import *

Using TensorFlow backend.


## Test data

In [2]:
dataset_length = 1000
D = 2
X = np.random.randn(dataset_length, D)*0.1
X[0:dataset_length//2, 0] += 0.1
X[0:dataset_length//2, 0] += 0.2
Y = np.ones(dataset_length)
Y[0:dataset_length//2] *= 0

X_test = np.random.randn(dataset_length, D)*0.1
X_test[0:dataset_length//2, 0] += 0.1
X_test[0:dataset_length//2, 0] += 0.2
Y_test = np.ones(dataset_length)
Y_test[0:dataset_length//2] *= 0

## Train a DT

In [4]:
clf = DecisionTreeClassifier(max_depth=5)
clf = clf.fit(X, Y)

In [30]:
a = NDTClassifier(D=2, gammas=[100, 1], tree_id=0)
a.compute_matrices_and_biases(clf)
a.to_keras(loss='binary_crossentropy', dropouts=[0., 0., 0.])
#print("FLOPs before:", a.count_ops)
h = a.fit(X, Y, epochs=10, sparse=False)

In [31]:
a.score(X_test, Y_test)

0.913

In [32]:
a.compute_weights_differences()

In [33]:
a.diff_W_in_nodes

,0_0,0_1,0_2,0_3,0_4,0_8,0_10,0_13,0_14,0_15,...,0_22,0_25,0_28,0_29,0_31,0_32,0_35,0_38,0_39,0_40
0,0.006253,0.034698,-0.000044,0.025483,-0.113118,0.049801,-0.046161,0.012316,-0.005273,-0.034998,...,-0.013601,0.001861,0.003419,0.002188,0.026669,-0.019476,-0.020577,-0.005701,0.048573,-0.016020
1,-0.000802,-0.046245,0.005666,0.041972,0.146443,-0.020756,0.009219,-0.018515,-0.000255,0.003797,...,-0.010203,0.022775,-0.006501,-0.005580,0.014745,0.013277,0.037198,-0.011392,0.008690,-0.027637


In [34]:
a.W_in_nodes

,0_0,0_1,0_2,0_3,0_4,0_8,0_10,0_13,0_14,0_15,...,0_22,0_25,0_28,0_29,0_31,0_32,0_35,0_38,0_39,0_40
0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0


In [35]:
a.W_in_nodes_nn

,0_0,0_1,0_2,0_3,0_4,0_8,0_10,0_13,0_14,0_15,...,0_22,0_25,0_28,0_29,0_31,0_32,0_35,0_38,0_39,0_40
0,0.993747,0.965302,1.000044,-0.025483,0.113118,0.950199,1.046161,0.987684,0.005273,0.034998,...,1.013601,-0.001861,0.996581,-0.002188,-0.026669,1.019476,0.020577,1.005701,0.951427,1.016020
1,0.000802,0.046245,-0.005666,0.958028,0.853557,0.020756,-0.009219,0.018515,1.000255,0.996203,...,0.010203,0.977225,0.006501,1.005580,0.985255,-0.013277,0.962802,0.011392,-0.008690,0.027637


In [36]:
w = a.W_nodes_leaves
idx = w.values.nonzero()
idx
w.values[idx] = 1

In [37]:
w.values[w.values.nonzero()]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1.])

In [38]:
a = NDTClassifier(D=2, gammas=[100, 1], tree_id=0)
a.compute_matrices_and_biases(clf)
a.to_keras(loss='binary_crossentropy', dropouts=[0., 0., 0.])
#print("FLOPs before:", a.count_ops)
h = a.fit(X, Y, epochs=10, sparse=True)

In [39]:
a.compute_weights_differences()

In [40]:
a.diff_W_in_nodes

,0_0,0_1,0_2,0_3,0_4,0_8,0_10,0_13,0_14,0_15,...,0_22,0_25,0_28,0_29,0_31,0_32,0_35,0_38,0_39,0_40
0,0.001869,-0.01391,0.03748,0.000000,0.00000,-0.000843,-0.025808,0.001359,0.000000,0.000000,...,-0.019994,0.000000,-0.027756,0.000000,0.000000,-0.011534,0.000000,-0.043362,-0.004111,-0.053046
1,0.000000,0.00000,0.00000,-0.022474,-0.02578,0.000000,0.000000,0.000000,0.005754,-0.018274,...,0.000000,0.009758,0.000000,-0.010398,-0.004397,0.000000,-0.026199,0.000000,0.000000,0.000000


In [41]:
a.W_in_nodes

,0_0,0_1,0_2,0_3,0_4,0_8,0_10,0_13,0_14,0_15,...,0_22,0_25,0_28,0_29,0_31,0_32,0_35,0_38,0_39,0_40
0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0


In [42]:
a.W_in_nodes_nn

,0_0,0_1,0_2,0_3,0_4,0_8,0_10,0_13,0_14,0_15,...,0_22,0_25,0_28,0_29,0_31,0_32,0_35,0_38,0_39,0_40
0,0.998131,1.01391,0.96252,0.000000,0.00000,1.000843,1.025808,0.998641,-0.000000,-0.000000,...,1.019994,0.000000,1.027756,-0.000000,0.000000,1.011534,0.000000,1.043362,1.004111,1.053046
1,-0.000000,-0.00000,-0.00000,1.022474,1.02578,0.000000,-0.000000,-0.000000,0.994246,1.018274,...,-0.000000,0.990242,0.000000,1.010398,1.004397,0.000000,1.026199,0.000000,0.000000,-0.000000


In [43]:
a.score(X_test, Y_test)

0.921

## Train an RF

In [3]:
clf = RandomForestClassifier(n_estimators=5, max_depth=5)
clf = clf.fit(X, Y)

In [4]:
a = NRFfullClassifier(D=2, gammas=[100, 1])
a.compute_matrices_and_biases(clf)
a.to_keras(loss='binary_crossentropy', dropouts=[0., 0., 0.])
#print("FLOPs before:", a.count_ops)
h = a.fit(X, Y, epochs=10, sparse=False)

In [8]:
a = NRFfullClassifier(D=2, gammas=[100, 1])
a.compute_matrices_and_biases(clf)
a.to_keras(loss='binary_crossentropy', dropouts=[0., 0., 0.])
#print("FLOPs before:", a.count_ops)
h = a.fit(X, Y, epochs=10, sparse=True)

In [9]:
a.compute_weights_differences()

In [11]:
a.diff_W_in_nodes

,0_0,0_1,0_2,0_4,0_33,0_38,0_7,0_8,0_9,0_12,...,4_5,4_10,4_11,4_12,4_14,4_17,4_19,4_22,4_24,4_26
0,-0.001325,-0.00349,-0.009133,-0.005222,0.008389,-0.014066,0.002209,0.000000,-0.002506,0.000517,...,0.022884,0.038506,-0.035315,0.029644,0.009846,0.000000,-0.007862,0.000000,0.000000,0.000000
1,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.029864,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.015521,0.000000,0.061836,0.030315,0.000916


In [12]:
a.W_in_nodes

,0_0,0_1,0_2,0_4,0_33,0_38,0_7,0_8,0_9,0_12,...,4_5,4_10,4_11,4_12,4_14,4_17,4_19,4_22,4_24,4_26
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0


In [13]:
a.W_in_nodes_nn

,0_0,0_1,0_2,0_4,0_33,0_38,0_7,0_8,0_9,0_12,...,4_5,4_10,4_11,4_12,4_14,4_17,4_19,4_22,4_24,4_26
0,1.001325,1.00349,1.009133,1.005222,0.991611,1.014066,0.997791,0.000000,1.002506,0.999483,...,0.977116,0.961494,1.035315,0.970356,0.990154,0.000000,1.007862,0.000000,0.000000,-0.000000
1,-0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,1.029864,-0.000000,-0.000000,...,0.000000,0.000000,0.000000,-0.000000,-0.000000,0.984479,-0.000000,0.938164,0.969685,0.999084


In [4]:
a = NRFindClassifier(D=2, gammas=[100, 1])
a.compute_matrices_and_biases(clf)
a.to_keras(loss='binary_crossentropy', dropouts=[0., 0., 0.])
#print("FLOPs before:", a.count_ops)
h = a.fit(X, Y, epochs=10, sparse=False)

In [7]:
a = NRFindClassifier(D=2, gammas=[100, 1])
a.compute_matrices_and_biases(clf)
a.to_keras(loss='binary_crossentropy', dropouts=[0., 0., 0.])
#print("FLOPs before:", a.count_ops)
h = a.fit(X, Y, epochs=10, sparse=True)

Model: "model_35"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 2)            0                                            
__________________________________________________________________________________________________
model_20 (Model)                (None, 2)            593         input_8[0][0]                    
__________________________________________________________________________________________________
model_23 (Model)                (None, 2)            335         input_8[0][0]                    
__________________________________________________________________________________________________
model_26 (Model)                (None, 2)            373         input_8[0][0]                    
___________________________________________________________________________________________

AttributeError: 'NRFindClassifier' object has no attribute 'W_in_nodes'

In [18]:
a.get_weights_from_NN()

9
